# This notebook will showcase extracting keyframes from videos

From [Wikipedia](https://en.wikipedia.org/wiki/Key_frame): In animation and filmmaking, a key frame (or keyframe) is a drawing or shot that defines the starting and ending points of a smooth transition. 

For running locally you will need the CLI tools.
```
sudo apt-get install ffmpeg imagemagick -y
```

We will use [ffmpeg](https://ffmpeg.org/ffmpeg.html) to extract keyframes from the video and imagemagick to combine the results. 


### Will keep trying to make something better, just adding extra steps to make the working clearer maybe??? 

In [ ]:
# First import the libraries we will use
import os # for handling files 
from subprocess import run # running shell scripts 
from IPython.display import Image, display # displaying results
import glob
import shutil

## Setup
Here you could make changes to suit your needs

In [ ]:
# Parameters (set these according to your environment and preferences)
input_videos = "videos"
output_dir = "keyframes_output/"
cutoff = 0.4          # Initial scene detection sensitivity
step = 0.8            # Sensitivity adjustment factor
iterations = 24       # Max attempts for adjustment
min_frames = 2        # Minimum keyframes per video we want
max_frames = 12       # Maximum keyframes per video we want


if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

## First let's try out ffmpeg on just one video 

In [ ]:
# Define the video file to process (ensure the file exists in the "videos" folder)
video_file = os.path.join(input_videos, "1.mov")  # Change to your video filename

# Create an output directory specific for this video’s keyframes
video_output_dir = os.path.join(output_dir, "video1")
os.makedirs(video_output_dir, exist_ok=True)

cutoff_here =0.02 # here I select a cutoff score that I know will work for this video in the full script this is found iteratively
# Build the ffmpeg command:
# -i : specifies the input file
# -vf : applies the video filter, here using the scene detection filter with the cutoff value
# -vsync vfr : forces variable frame rate output (only the detected frames are saved)
# The output images are named frame_001.jpg, frame_002.jpg, etc.
ffmpeg_command = f'''ffmpeg -i "{video_file}" -vf "select=gt(scene\\,{cutoff_here})" -vsync vfr "{video_output_dir}/frame_%03d.jpg"'''
print("Running:", ffmpeg_command)

# Run the ffmpeg command
result = run(ffmpeg_command, shell=True)

# List the extracted keyframes
extracted_frames = sorted(glob.glob(os.path.join(video_output_dir, "*.jpg")))
display(Image(extracted_frames[0]))

So with shutil we are able to pull frames out of the video, but we don't know how much "action" there is in the video. That is we need a way to set the cutoff score in a sensible way for each video. 

## Creating the functions to process the videos

In [ ]:
# This function calls ffmpeg and extracts the keyframes
def keyframes(input_file, output_dir, cutoff, iterations):
    file_name = os.path.splitext(os.path.basename(input_file))[0]  # create a file_name based on the video being processed
    # some error handling 
    if iterations == 0:
        print("ERROR: Stopped trying; increase iterations parameter to try more!")
        return None
    # Build the output pattern using os.path.join
    output_pattern = os.path.join(output_dir, f"{file_name}-%02d.jpg")
    # run ffmpeg and save the result
    command = f'''ffmpeg -i "{input_file}" -vf "select='gt(scene,{cutoff})'" -vsync 0 "{output_pattern}" '''
    print(f"Running: {command}")
    result = run(command, shell=True, capture_output=True)
    # If we see an error print it and try again, else check how many frames we have
    if result.returncode != 0:
        print(f"ffmpeg error: {result.stderr.decode().strip()}")
        return keyframes(input_file, output_dir, cutoff * step, iterations - 1) 
    else:
        return adaptive_keyframes(input_file, output_dir, file_name, cutoff, iterations)

# This code adapts the cutoff value to find the optimal one for the video
# This way we find the optimal diversity of frames within each video
def adaptive_keyframes(input_file, output_dir, file_name, cutoff, iterations):
    # Check how many frames we have found
    frames = [f for f in os.listdir(output_dir) if f.startswith(file_name) and f.endswith(".jpg")]
    n = len(frames)
    print(f"Found {n} keyframes for {file_name}.")
    
    # If we have too few frames we decrease the cutoff making ffmpeg more sensitive
    # If we have too many frames we make ffmpeg less sensitive 
    # Once we found a number of frames with ffmpeg between the maximum and minimum number we defined we combine them
    if n < min_frames:
        print("Too few keyframes, let's try again!")
        return keyframes(input_file, output_dir, cutoff * step, iterations - 1)
    elif n > max_frames:
        print("Too many keyframes, let's try again!")
        return keyframes(input_file, output_dir, cutoff * (1 + step/2), iterations - 1)
    else:
        print(f"Solution found! Check {output_dir} directory.")
        return combine_keyframes(input_file, output_dir, file_name)

# This function combines our resulting frames into a single image using magick 
def combine_keyframes(input_file, output_dir, file_name):
    # first we create a unique name for our result
    combined_img = os.path.join(output_dir, f"{file_name}-keyframes.jpg")
    # then we can call magic to concat the images 
    command = f'''convert +append "{output_dir}/{file_name}-*.jpg" "{combined_img}" '''
    print(f"Combining keyframes: {command}")
    result = run(command, shell=True)
    # tabulate the result and return the image
    if result.returncode == 0:
        print(f"Keyframes have been saved in a single file under: {combined_img}")
        display(Image(combined_img))
        return combined_img
    else:
        print("Error combining keyframes. Make sure ImageMagick is installed.")
        return None

## Now let's process our videos by looping through the folder with our videos

In [ ]:
for video_file in os.listdir(input_videos): # loop throug each file in the folder
    # create a path variable to each video
    input_video_path = os.path.join(input_videos, video_file)
    # Create a subdirectory for the output for each video to avoid mixing outputs
    video_output_dir = os.path.join(output_dir, os.path.splitext(video_file)[0])
    os.makedirs(video_output_dir, exist_ok= True)
    # process each video using our functions
    print(f"\nProcessing video: {input_video_path}")
    keyframes(input_file=input_video_path, output_dir=video_output_dir, cutoff=cutoff, iterations=iterations)


## At last let's create a meta image from the extracted keyframes

In [ ]:

combined_image_paths = glob.glob(os.path.join("keyframes_output", "*", "*-keyframes.jpg"))
# Join them into a single space-separated string
combined_paths = " ".join(combined_image_paths)

# Set the final output file for all movies combined
final_combined_image = "combined_all_movies.jpg"

combine_command = f"convert -append {combined_paths} {final_combined_image}" # note here we use -append to append vertically
print(f"Running: {combine_command}")
result = run(combine_command, shell=True)

display(Image(final_combined_image))


Now we can interpret the results and have an overview of many videos most different frames at once. 